In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

**Подготовка исходных данных**

In [7]:
X_init = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

Стандартизация

In [8]:
X = X.copy()
X[:, 1] = standard_scale(X[:, 1])
X[:, 2] = standard_scale(X[:, 2])
X[:, 3] = standard_scale(X[:, 3])

# Задачи

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

Исходная функция

In [9]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

Доработанная функция

Вариант 1. через цикл

In [11]:
def calc_logloss_loop(y, y_pred):
    total = []
    for i in range (y.shape[0]):
        if y_pred[i] == 1.0:
            err = 0
        elif y_pred[i] == 0:
            err = None    
        else:
            err = -(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
        
        total.append(err)
    return np.array(total).mean()

Вариант 2. Замена чисел в массиве на значимые величины отлиичные от 0 или 1, но максимально приближенные к ним.

In [12]:
def calc_logloss_exchange(y, y_pred):
    y_pred_cp = y_pred.copy()
    y_pred_cp[y_pred_cp==0]=1e-17
    y_pred_cp[y_pred_cp==1]=1-(1e-17)
    err = - np.mean(y * np.log(y_pred_cp) + (1.0 - y) * np.log(1.0 - y_pred_cp))
    return err

Проверка работы функции

In [22]:
def eval_model(X, y, iterations, eta):
    w_old = np.inf
    epsilon = 1e-9
    data = []
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z) # произведение переводится в сигмоид 0-1
        err = calc_logloss_loop(y, y_pred)        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ        
        data.append(err)
        if np.linalg.norm(W - w_old, ord=2) < epsilon:
            break
    return W, data

In [23]:
eta = 1e-4
iterations = 5000

In [24]:
W, losses_ex = eval_model(X_st, y, iterations, eta)

In [25]:
np.array(losses_ex).min()

0.6939557095083029

Функции вне зависимости о реализации возвращают одинаковые результаты.

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [26]:
def calc_logloss_exchange(y, y_pred):
    y_pred_cp = y_pred.copy()
    y_pred_cp[y_pred_cp==0]=1e-17
    y_pred_cp[y_pred_cp==1]=1-(1e-17)
    err = - np.mean(y * np.log(y_pred_cp) + (1.0 - y) * np.log(1.0 - y_pred_cp))
    return err

Функция eval_model  
единственная доработка функции, которая была произведена - добавление значения err к выводу функции вместо возвращаемых весов, т.к. именно по ошибке будет производится измерение

In [27]:
def eval_model(X, y, iterations, eta):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss_exchange(y, y_pred)
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        #if i % (iterations / 100) == 0:
        #    print(i, W, err)
    return  err #, W

Т.к. X и y являются неизменяемыми, подбор может вестить только по двум параметрам - количеству итераций и величине eta.

In [28]:
eta_arr=np.logspace(-5, -1, 25)
iterations_arr = np.linspace(500, 10000, 20, dtype='int16')

In [30]:
iterations_arr

array([  500,  1000,  1500,  2000,  2500,  3000,  3500,  4000,  4500,
        5000,  5500,  6000,  6500,  7000,  7500,  8000,  8500,  9000,
        9500, 10000], dtype=int16)

In [29]:
eta_arr

array([1.00000000e-05, 1.46779927e-05, 2.15443469e-05, 3.16227766e-05,
       4.64158883e-05, 6.81292069e-05, 1.00000000e-04, 1.46779927e-04,
       2.15443469e-04, 3.16227766e-04, 4.64158883e-04, 6.81292069e-04,
       1.00000000e-03, 1.46779927e-03, 2.15443469e-03, 3.16227766e-03,
       4.64158883e-03, 6.81292069e-03, 1.00000000e-02, 1.46779927e-02,
       2.15443469e-02, 3.16227766e-02, 4.64158883e-02, 6.81292069e-02,
       1.00000000e-01])

Перебор значений в цикле для разных вариантов исходной величины шага и количества итерраций

In [31]:
values = []
for iterations in iterations_arr:
    for eta_elem in eta_arr:
        err = eval_model(X, y, iterations, eta)
        values.append([err, iterations, eta])

In [32]:
values

[[0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.7537963671147336, 500, 0.0001],
 [0.746724940751059, 1000, 0.0001],
 [0.746724940751059, 1000, 0.0001],
 [0.746724940751059, 1000, 0

In [33]:
values_np = np.array(values)

In [34]:
min_loss = values_np[:, 0].min()
min_loss

0.6374935132205644

In [35]:
iterations_number = values_np[values_np[: ,0] == min_loss][0][1]
iterations_number

10000.0

In [36]:
eta_val = values_np[values_np[: ,0] == min_loss][0][2]
eta_val

0.0001

Т.к. список содержит больше одной величины, с одинаковым уровнем потерь, выберем самую верзнюю строчку изз списка. Т.е. минимальное количество итераций и, что вполне логично, одна из наименьших величин шага

**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**


Сигмоидная функция без дополнительных преобразований выдает вероятность, что  элемент относится к классу 1.
Т.е. на выходе доработанная функция calc_pred_proba должны выдавать выход симоидной функции.

In [37]:
def calc_pred_proba(W, X, threshold = 0.5):
    z = np.dot(X, W)
    print(z)
    y_pred_proba = sigmoid(z) 
    return y_pred_proba

Проверка работы функции.  
При наличии серьезных дисбалансов без нормализации/стандартизации выдаваемый результат может получается искаженным.
В качестве X использован стандартизированный массив.

In [39]:
a = calc_pred_proba(W, X)
a

[-1.17209193 -0.95114871  0.88528486 -1.44256352  1.56335032 -0.7454413
  3.30454809  3.38072713 -1.22732773  1.16146388]


array([0.23647707, 0.27865387, 0.70791618, 0.19114869, 0.82683358,
       0.32181543, 0.96458451, 0.96709675, 0.22664948, 0.76159861])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

Добавлена отдельная функция, которая оценивает элемент на соответствие указанному попрогу и после возвращает уже посчитанную величину класса

In [40]:
def rule(elem):
    threshold = 0.5
    if elem > threshold:
        return 1
    return 0    

In [41]:
def calc_pred(W, X):
    z = np.dot(X, W)
    print(z)
    y_pred_proba = sigmoid(z)
    y_pred = np.array(list(map(rule, y_pred_proba)))
    return y_pred
    

Проверка работы функции.   
в качестве одного из промежуточных результатов на печать выводиться произведение W, X

In [42]:
a = calc_pred(W, X)
a

[-1.17209193 -0.95114871  0.88528486 -1.44256352  1.56335032 -0.7454413
  3.30454809  3.38072713 -1.22732773  1.16146388]


array([0, 0, 1, 0, 1, 0, 1, 1, 0, 1])

**5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

Добавим массив предсказанных величин y_pred для оценки работоспособности функций:

In [43]:
y_pred = calc_pred(W, X)
y_pred

[-1.17209193 -0.95114871  0.88528486 -1.44256352  1.56335032 -0.7454413
  3.30454809  3.38072713 -1.22732773  1.16146388]


array([0, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [44]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

Accuracy. Вычисляется как отношение количества правильных результатов ко всему общему количеству измерений.
$$accuracy = \frac{TP+TN}{TP+FP+TN+FN}.$$

In [45]:
def accuracy(y, y_pred):
    res = 0
    if y.shape[0] != y_pred.shape[0]:
        return None
    for i in range (y.shape[0]):
        if y[i] == y_pred[i]:
            res += 1
    n = y.shape[0]
    value = res/n
    
    return value

In [46]:
accuracy(y, y_pred)

0.8

In [47]:
train_accuracy = np.mean(y_pred == y) * 100.0
train_accuracy

80.0

$$precision(a, X) = \frac{TP}{TP+FP}.$$

In [48]:
def precision(y, y_pred):
    tp = 0
    fp = 0
    
    if y.shape[0] != y_pred.shape[0]:
        return None
    
    for i in range (y.shape[0]):
        if y[i] == 1 and y_pred[i]==y[i]:
            tp +=1
        elif y[i] == 0 and y_pred[i]== 1:
            fp +=1
    value = tp/(tp + fp)
    return value

In [49]:
precision(y, y_pred)

0.8

$$recall(a, X) = \frac{TP}{TP+FN},$$

In [50]:
def recall(y, y_pred):
    tp = 0
    fn = 0
    
    if y.shape[0] != y_pred.shape[0]:
        return None
    
    for i in range (y.shape[0]):
        if y[i] == 1 and y_pred[i]==y[i]:
            tp +=1
        elif y[i] == 1 and y_pred[i]== 0:
            fn +=1
    value = tp/(tp + fn)
    return value

In [51]:
recall(y, y_pred)

0.8

$$F = \frac{2 \cdot precision \cdot recall }{ presision + recall}.$$

In [52]:
def f_score(y, y_pred):
    prc = precision(y, y_pred)
    rc = recall(y, y_pred)
    if prc is not None and rc is not None:
        F = (2 * prc * rc)/(prc + rc)
    else:
        return None
    return F

In [53]:
f_score(y, y_pred)

0.8000000000000002

**6. Могла ли модель переобучиться? Почему?**

Да. Как и практически любая модель в машинном обучении эта может переобучится.
Суть данной модели близка к линейной регрессии при том же самом алгоритме подбора весов при признаках.
Возможные причины переобучения для данной модели:
- маленький набор исходных данных. 10 наборов наблюдений - это очень мало.
- вероятен дополнительный шум, который имеет большое значение (опять же связано с очень маленьким размером выборки)
- формулы в текущей модели не имеют других критериев выхода, кроме количества итерраций, т.е. возможна чересчур точная подгонка коэффициентов под исходных данных, особенно при маленькой величине шага.
- опять же, из-за маленкього датасета практичски невозможно утроить кроссвалидацию для определения наличия факта переобучения.

